## Machine learning on EEG data

This tutorial walks through some basic tools for applying ML techniques on EEG data through MNE

### 1. Importing and preprocessing data

In [ ]:
import mne

# Folder & files containing the data:
data_path = '/Users/athina/Documents/Bremen2018/PracticalPrep/'
data_file = '817_1_PDDys_ODDBALL_Clean_curated'

filename = data_path + data_file

# We read the EEG epochs:
epochs = mne.read_epochs(filename + '.fif')

epochs = epochs['Standard', 'Novel']

epochs.filter(l_freq = 0.1, h_freq = 20)
epochs.apply_baseline((None, 0))

The main information we wish to retain is (a) the data (b) the true labels

In [ ]:
# and get the data:
data = epochs._data

# and labels:
labels = epochs.events[:,-1]

##### Reminder exercise: how many trials of each condition do we have?

In [ ]:
epochs.event_id

### 2. Using sklearn to classify EEG data
We can now proceed with some mne.decoding and sklearn functionalities:

In [ ]:
from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import train_test_split

'make_pipeline' will create our analysis pipeline from a set of estimators, sklearn-style.
'Vectorizer' comes from MNE and will pull together all epochs x time x channels data to a vector of samples x channels
'StandardScaler' normalizes the data by their median

In [ ]:
clf = make_pipeline(Vectorizer(), StandardScaler(),
                    svm.SVC(kernel='linear',C=1)
                   )

We can then use scikit-learn function 'train_test_split' to obtain a random split of training and test datasets:

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.4, random_state=0)

##### Exercise: What are the dimensions of the train / test datasets?

In [ ]:
labels_train.shape

Now we can treat 'clf' as any sklearn estimator, and fit the training data to the classifier:

In [ ]:
clf.fit(data_train, labels_train)

And we can then use the classifier to predic the labels of the test data:

In [ ]:
clf.predict(data_train)

Or, we can directly compute the classification score:

In [ ]:
clf.score(data_train,labels_train)

##### Exercise: What is the classification score on the test dataset? What do you observe?

In [ ]:
clf.score(data_test,labels_test)

Alternatively, we can also compute the classification accuracy based on cross-validation. By default, the selected model will be fitted on 3 different splits of the data:

In [ ]:
clf_CV = make_pipeline(Vectorizer(), StandardScaler(),
                    svm.SVC(kernel='linear',C=1)
                   )

cross_val_score(clf_CV, data, labels)

We can change the number of CV folds by including an additional parameter, 'cv':

Remember that for integer or None inputs, if the estimator is a classifier and the labels are binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used (i.e. the folds are made by preserving the percentage of samples for each class):

In [ ]:
scores = cross_val_score(clf_CV, data, labels, cv=5)
print(scores)

And we can now compute the mean and 95% confidence interval of the classification accuracy:

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

##### Exercise: What happens to the mean accuracy if you increase the CV folds? If you decrease them?

In [ ]:
scores = cross_val_score(clf_CV, data, labels, cv=10)
print(scores)

Additionally, we can evaluate multiple metrics for our models at once, like the time it takes to fit them, the train and test scores with the function 'cross_validate':

In [ ]:
from sklearn.model_selection import cross_validate

cross_validate(clf, data, labels)

Now we will first apply PCA to our data and then classify them. What do you observe?

In [ ]:
from mne.decoding import UnsupervisedSpatialFilter

from sklearn.decomposition import PCA

pca = UnsupervisedSpatialFilter(PCA(n_components = 30)) # we create an instance of PCA

In [ ]:
pca_data = pca.fit_transform(data)

Get the explained variance for the retained components:

In [ ]:
pca.estimator.explained_variance_

Now, we classify the PCA-transformed data:

In [ ]:
cross_validate(clf, pca_data, labels)

What do you observe?

### 3. Optimizing our classifier

In the above paradigm we just used a default implementation of SVM with a linear kernel and a hyperparameter C set to 1. Now we will estimate the parameters of SVM through exhaustive Grid Search.

'GridSearchCV' exhaustively tests candidate models from a grid of parameter values specified in the following form:

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ['linear', 'rbf'],'C':[5, 10]}

Now we can use GridSearchCV directly in 'make_pipeline' to proceed with an optimized model:

In [ ]:
clf_opt2 = make_pipeline(Vectorizer(), StandardScaler(), GridSearchCV(svm.SVC(), parameters,cv=5))
print(clf_opt2)
tmp = clf_opt2.fit(data, labels)
print(tmp)
cross_val_score(clf_opt2,data,labels)

What do you observe?

You can assess the optimized parameters through:

In [ ]:
tmp = clf_opt2.steps[-1][1]
tmp.best_params_

The classifier that you trained & optimized above may still be biased. Any ideas why?

##### Exercise: how could you structure your dataset to make it unbiased?

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.5, random_state=0)

In [ ]:
clf_opt2 = make_pipeline(Vectorizer(), StandardScaler(), GridSearchCV(svm.SVC(), parameters,cv=5))
clf_opt2.fit(data_train, labels_train)

In [ ]:
import numpy as np

clf_opt2.steps[-1]
perf = cross_val_score(clf_opt2,data_test,labels_test, cv = 5)
np.mean(perf)

### 4. Making a more sophisticated pipeline

In the steps above we were still evaluating a rather simplified version of our data, where we considered EEG data of one condition and the other irrespective of time. Now we will take the temporal dimension into account as well and will evaluate how classification accuracy changes across time.

For this we will use 'SlidingEstimator' from MNE:

In [ ]:
from mne.decoding import SlidingEstimator, cross_val_multiscore

clf = make_pipeline(Vectorizer(), StandardScaler(),
                    svm.SVC(kernel='linear',C=1)
                   )

sl = SlidingEstimator(clf) # we apply the sliding estimator to 'clf'

CV_score_time = cross_val_multiscore(sl, data, labels)

What are the dimensions of the classification score now? Why?

In [ ]:
CV_score_time.shape

We can now plot the classification accuracy across time:

In [ ]:
import matplotlib.pyplot as plt

%matplotlib tk

fig, ax = plt.subplots()
ax.plot(epochs.times, CV_score_time.T)
plt.show()

##### Exercise: Can you now plot the mean classification accuracy over the CV folds as a function of time?

In [ ]:
fig, ax = plt.subplots()
ax.plot(epochs.times, np.mean(CV_score_time,axis = 0))
plt.show()

Another important steps of classifying EEG data is to extract and visualize the features of our classifier. In the following case, we will extract the coefficients of SVM across time and plot them as topographic maps:

In [ ]:
import numpy as np

topos = np.array([svm.SVC(kernel='linear').fit(time_point.T, labels).coef_ * time_point.std(1)
                  for time_point in data.T])[:, 0, :]
topo_ev = mne.EvokedArray(topos.T, info=epochs.info, tmin=-.1, nave=len(labels))
topo_ev.plot_joint(times=[.22, .3, .375, .45]);

##### Exercise: Can you now plot the classification coefficients on the time-points of maximal classification accuracy?

### 5. Implementing a more sophisticated pipeline

In the same pipeline we can additionally integrate dimensionality reduction techniques. How?